In [ ]:
import os
import sys
import numpy as np
import seaborn as sns
from sklearn import metrics

root_dir = "../"
sys.path.append(root_dir)
import configs
import datasets
from ibydmt.utils.concepts import get_concepts
from ibydmt.utils.config import get_config
from ibydmt.utils.data import get_dataset
from ibydmt.utils.result import TestingResults
from ibydmt.tester import get_local_test_idx

config_name, concept_type = "cub", "image"
results_kw = {"testing.kernel_scale": 0.5, "testing.tau_max": 200}

config = get_config(config_name)
backbone_configs = config.sweep(["data.backbone"])

dataset = get_dataset(config)
classes = dataset.classes

test_idx = get_local_test_idx(config)
n_test_images = sum([len(v) for v in test_idx.values()])

sns.set_style()
sns.set_context("paper")

In [ ]:
cardinalities = config.testing.cardinality
xskit_f1 = np.zeros((len(backbone_configs), len(cardinalities), n_test_images))
for i, backbone_config in enumerate(backbone_configs):
    backbone = backbone_config.data.backbone

    backbone_results = TestingResults.load(
        backbone_config, "local_cond", concept_type, results_kw=results_kw
    )

    j = 0
    for class_name, class_test_idx in test_idx.items():
        for image_idx in class_test_idx:
            _, concepts = get_concepts(config, concept_image_idx=image_idx)
            m = len(concepts)
            gt_importance = [1] * (m // 2) + [0] * (m // 2)

            for k, cardinality in enumerate(cardinalities):
                sorted_results = backbone_results.sort(
                    class_name, image_idx=image_idx, cardinality=cardinality
                )
                sorted_concepts = sorted_results["sorted_concepts"]
                sorted_importance = sorted_results["sorted_importance"]

                sorted_gt_importance = [
                    gt_importance[concepts.index(c)] for c in sorted_concepts
                ]
                xskit_f1[i, k, j] = metrics.f1_score(
                    sorted_gt_importance, sorted_importance
                )

            j += 1

    print(
        f"Backbone: {backbone}:\n\t cardinality: {cardinalities[0]}, f1:"
        f" {np.mean(xskit_f1[i, 0]):.2f} pm {np.std(xskit_f1[i, 0]):.2f}\n\t"
        f" cardinality: {cardinalities[1]}, f1: {np.mean(xskit_f1[i, 1]):.2f} pm"
        f" {np.std(xskit_f1[i, 1]):.2f}\n\t cardinality: {cardinalities[2]}, f1:"
        f" {np.mean(xskit_f1[i, 2]):.2f} pm {np.std(xskit_f1[i, 2]):.2f}\n"
    )

In [ ]:
print(
    f"Average:\n\t cardinality: {cardinalities[0]}, f1:"
    f" {np.mean(xskit_f1[:, 0]):.2f} pm {np.std(xskit_f1[:, 0]):.2f}\n\t cardinality:"
    f" {cardinalities[1]}, f1: {np.mean(xskit_f1[:, 1]):.2f} pm"
    f" {np.std(xskit_f1[:, 1]):.2f}\n\t cardinality: {cardinalities[2]}, f1:"
    f" {np.mean(xskit_f1[:, 2]):.2f} pm {np.std(xskit_f1[:, 2]):.2f}"
)